# QuartzNet finetuning

In [ ]:
## Set NeMo branch
BRANCH = "r1.1.0"

In [ ]:
try:
  from google import colab
  COLAB_ENV = True
except (ImportError, ModuleNotFoundError):
  COLAB_ENV = False

In [ ]:
if COLAB_ENV:
  %%capture
  !git clone https://github.com/TheSoundOfAIOSR/rg_speech_to_text
  !pip install wget
  !apt-get install sox libsndfile1 ffmpeg libsox-fmt-mp3
  !pip install unidecode
  !pip install matplotlib>=3.3.2
  !pip install torchaudio
  !pip install jiwer
  ## Install NeMo (use BRANCH variable)
  !python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]
  """
  Remember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!
  Alternatively, you can uncomment the exit() below to crash and restart the kernel, in the case
  that you want to use the "Run All Cells" (or similar) option.
  """
  # exit() 

# Installing Dependencies

In [ ]:
import os
import glob
import subprocess
import tarfile
import wget
import copy
from tqdm import tqdm
from omegaconf import OmegaConf, open_dict

In [ ]:
data_dir = 'datasets/'

if not os.path.exists(data_dir):
  os.makedirs(data_dir, exist_ok=True)

if not os.path.exists("scripts"):
  os.makedirs("scripts")

In [ ]:
import nemo
import nemo.collections.asr as nemo_asr
from nemo.collections.asr.metrics.wer import word_error_rate
from nemo.utils import logging, exp_manager

# Loading and preprocessing data

In [ ]:
if COLAB_ENV:
    project_root = "rg_speech_to_text"
else:
    project_root = ".."
csv_path = project_root + "/data/finetuning-dataset/finetuning-dataset.csv"
audio_path = project_root + "/data/finetuning-dataset/audiofiles"
file_prefixes = ['TA-', 'TK-', 'TM-']

In [ ]:
# Dataset file reading/writing utils

def match_id_to_audiofile(audio_path, id, prefixes, extension='.wav'):
  fnames = [os.path.join(os.path.abspath(audio_path), prefix + id + extension) for prefix in prefixes]
  fexists = [f for f in fnames if os.path.exists(f)]
  return fexists

def to_manifest_dict(audiofiles, text):
  return [{'audio_filepath': af, 'duration': 3.000, 'text': text} for af in audiofiles]

def read_datasetcsv(csv_path, audio_path, prefixes):
  manifest = []
  with open(csv_path, 'r') as f:
    for line in tqdm(f, desc="Reading dataset"):
      line = line.replace("\n", "")
      data = line.split("|")
      manifest.append(to_manifest_dict(
          match_id_to_audiofile(audio_path, data[0], prefixes), data[1]))
  return [item for sublist in manifest for item in sublist]

def read_manifest(path):
    manifest = []
    with open(path, 'r') as f:
        for line in tqdm(f, desc="Reading manifest data"):
            line = line.replace("\n", "")
            data = json.loads(line)
            manifest.append(data)
    return manifest

def write_processed_manifest(data, original_path, category):
  original_manifest_name = os.path.basename(original_path)
  new_manifest_name = original_manifest_name.replace(".csv", "_" + category + "_processed.json")

  manifest_dir = os.path.split(original_path)[0]
  filepath = os.path.join(manifest_dir, new_manifest_name)
  with open(filepath, 'w') as f:
    for datum in tqdm(data, desc="Writing manifest data"):
      datum = json.dumps(datum)
      f.write(f"{datum}\n")
  print(f"Finished writing manifest: {filepath}")
  return filepath

In [ ]:
import numpy as np
import pandas as pd

def train_validate_test_split(df, train_percent=.6, validate_percent=.2, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [ ]:
mixed_manifest_data = read_datasetcsv(csv_path, audio_path, file_prefixes)

In [ ]:
def split_mixed_manifest_data(mixed_manifest_data):
    train, dev, test = train_validate_test_split(pd.DataFrame(mixed_manifest_data))
    return train.to_dict('records'), dev.to_dict('records'), test.to_dict('records')

In [ ]:
train_manifest_data, dev_manifest_data, test_manifest_data = split_mixed_manifest_data(mixed_manifest_data)

In [ ]:
train_text = [data['text'] for data in train_manifest_data]
dev_text = [data['text'] for data in dev_manifest_data]
test_text = [data['text'] for data in test_manifest_data]

## Character set

In [ ]:
from collections import defaultdict

def get_charset(manifest_data):
    charset = defaultdict(int)
    for row in tqdm(manifest_data, desc="Computing character set"):
        text = row['text']
        for character in text:
            charset[character] += 1
    return charset

In [ ]:
train_charset = get_charset(train_manifest_data)
dev_charset = get_charset(dev_manifest_data)
test_charset = get_charset(test_manifest_data)

In [ ]:
train_dev_set = set.union(set(train_charset.keys()), set(dev_charset.keys()))
test_set = set(test_charset.keys())

Count the number of unique tokens that exist within this dataset

In [ ]:
print(f"Number of tokens in train+dev set : {len(train_dev_set)}")
print(f"Number of tokens in test set : {len(test_set)}")

## Process special character tokens

In [ ]:
# Preprocessing
import re

chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\…\{\}\【\】\・\。\『\』\、\ー\〜]'  # remove special character tokens

def remove_special_characters(data):
    data["text"] = re.sub(chars_to_ignore_regex, '', data["text"]).lower().strip()
    return data

def change_digit_to_word(data):
  data["text"] = data["text"].replace("0", "zero ")
  data["text"] = data["text"].replace("1", "one ")
  data["text"] = data["text"].replace("2", "two ")
  data["text"] = data["text"].replace("3", "three ")
  data["text"] = data["text"].replace("4", "four ")
  data["text"] = data["text"].replace("5", "five ")
  data["text"] = data["text"].replace("6", "six ")
  data["text"] = data["text"].replace("7", "seven ")
  data["text"] = data["text"].replace("8", "eight ")
  data["text"] = data["text"].replace("9", "nine ")
  return data

def replace_double_spaces(data):
  data["text"] = data["text"].replace("  ", " ")
  return data

# Processing pipeline
def apply_preprocessors(manifest, preprocessors):
    for processor in preprocessors:
        for idx in tqdm(range(len(manifest)), desc=f"Applying {processor.__name__}"):
            manifest[idx] = processor(manifest[idx])

    print("Finished processing manifest !")
    return manifest

# List of pre-processing functions
PREPROCESSORS = [
    remove_special_characters,
    change_digit_to_word,
    replace_double_spaces,
]


In [ ]:
# Apply preprocessing
train_data_processed = apply_preprocessors(train_manifest_data, PREPROCESSORS)
dev_data_processed = apply_preprocessors(dev_manifest_data, PREPROCESSORS)
test_data_processed = apply_preprocessors(test_manifest_data, PREPROCESSORS)

In [ ]:
# Write new manifests
import json
train_manifest_cleaned = write_processed_manifest(train_data_processed, csv_path, 'train')
dev_manifest_cleaned = write_processed_manifest(dev_data_processed, csv_path, 'dev')
test_manifest_cleaned = write_processed_manifest(test_data_processed, csv_path, 'test')


In [ ]:
train_manifest_data = read_manifest(train_manifest_cleaned)
train_charset = get_charset(train_manifest_data)

dev_manifest_data = read_manifest(dev_manifest_cleaned)
dev_charset = get_charset(dev_manifest_data)

train_dev_set = set.union(set(train_charset.keys()), set(dev_charset.keys()))

In [ ]:
print(f"Number of tokens in preprocessed train+dev set : {len(train_dev_set)}")

# Character Encoding CTC Model

Now that we have a processed dataset, we can begin training an ASR model on this dataset. The following section will detail how we prepare a CTC model which utilizes a Character Encoding scheme.

This section will utilize a pre-trained [QuartzNet 15x5](https://arxiv.org/abs/1910.10261), which has been trained on roughly 7,000 hours of English speech base model. We will modify the decoder layer (thereby changing the model's vocabulary) and then train for a small number of epochs.

In [ ]:
# "stt_en_quartznet15x5"
char_model = nemo_asr.models.ASRModel.from_pretrained("stt_en_quartznet15x5", map_location='cpu')

## Update the vocabulary

Changing the vocabulary of a character encoding ASR model is as simple as passing the list of new tokens that comprise the vocabulary as input to `change_vocabulary()`.

In [ ]:
char_model.change_vocabulary(new_vocabulary=list(train_dev_set))

## Training on domain specific text

It is also important to note that if the language remains the same, and some specific domain of text must be adapted for ASR, it is often easier to add a domain-specific language model to guide the generic ASR model than to attempt fine-tuning a full ASR model on limited data from that specific domain. But we would like to train on dialects we speak, therefore we train on audio files, exactly as other models used in the project.


In [ ]:
#@title Freeze Encoder { display-mode: "form" }
freeze_encoder = True #@param ["False", "True"] {type:"raw"}
freeze_encoder = bool(freeze_encoder)

### Frozen Encoder - Unfrozen Batch Normalization

Freezing the encoder is generally helpful to limit computation and enable faster training; however, in many experiments, freezing the encoder in its entirety will often prevent a model from learning on low-resource languages. 

In order to enable a frozen encoder model to learn on a new language stably, we, therefore, unfreeze the batch normalization layers in the encoder. On top of this, if the model contains "SqueezeExcite" sub-modules, we unfreeze them as well.

In doing so, we notice that such models train properly and obtain respectable scores even on severely resource-limited languages.

------

**Note**: This phenomenon disappears when sufficient data is available (in such a case, the entire encoder can be trained as well). Therefore it is advised to unfreeze the encoder when sufficient data is available.

In [ ]:
import torch
import torch.nn as nn

def enable_bn_se(m):
    if type(m) == nn.BatchNorm1d:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

    if 'SqueezeExcite' in type(m).__name__:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

In [ ]:
if freeze_encoder:
  char_model.encoder.freeze()
  char_model.encoder.apply(enable_bn_se)
  logging.info("Model encoder has been frozen, and batch normalization has been unfrozen")
else:
  char_model.encoder.unfreeze()
  logging.info("Model encoder has been un-frozen")

## Update config

Each NeMo model has a config embedded in it, which can be accessed via `model.cfg`. In general, this is the config that was used to construct the model.

For pre-trained models, this config generally represents the config used to construct the model when it was trained. A nice benefit to this embedded config is that we can repurpose it to set up new data loaders, optimizers, schedulers, and even data augmentation!

In [ ]:
char_model.cfg.labels = list(train_dev_set)

Now, we create a working copy of the model config and update it as needed.

In [ ]:
cfg = copy.deepcopy(char_model.cfg)

### Setting up data loaders

Now that the model's character set has been updated let's prepare the model to utilize the new character set even in the data loaders. Note that this is crucial so that the data produced during training/validation matches the new character set, and tokens are encoded/decoded correctly.

**Note**: An important config parameter is `normalize_transcripts` and `parser`. There are some parsers that are used for specific languages for character based models - currently only `en` is supported. These parsers will preprocess the text with the given languages parser. However, for other languages, it is advised to explicitly set `normalize_transcripts = False` - which will prevent the parser from processing text. 

In [ ]:
# Setup train, validation, test configs
with open_dict(cfg):    
  # Train dataset  (Concatenate train manifest cleaned and dev manifest cleaned)
  cfg.train_ds.manifest_filepath = f"{train_manifest_cleaned},{dev_manifest_cleaned}"
  cfg.train_ds.labels = list(train_dev_set)
  cfg.train_ds.normalize_transcripts = False
  cfg.train_ds.batch_size = 32
  cfg.train_ds.num_workers = 8
  cfg.train_ds.pin_memory = True
  cfg.train_ds.trim_silence = True

  # Validation dataset  (Use test dataset as validation, since we train using train + dev)
  cfg.validation_ds.manifest_filepath = test_manifest_cleaned
  cfg.validation_ds.labels = list(train_dev_set)
  cfg.validation_ds.normalize_transcripts = False
  cfg.validation_ds.batch_size = 8
  cfg.validation_ds.num_workers = 8
  cfg.validation_ds.pin_memory = True
  cfg.validation_ds.trim_silence = True

In [ ]:
# setup data loaders with new configs
char_model.setup_training_data(cfg.train_ds)
char_model.setup_multiple_validation_data(cfg.validation_ds)

### Setting up optimizer and scheduler

When fine-tuning character models, it is generally advised to use a lower learning rate and reduced warmup. A reduced learning rate helps preserve the pre-trained weights of the encoder. Since the fine-tuning dataset is generally smaller than the original training dataset, the warmup steps would be far too much for the smaller fine-tuning dataset.

-----
**Note**: When freezing the encoder, it is possible to use the original learning rate as the model was trained on. The original learning rate can be used because the encoder is frozen, so the learning rate is used only to optimize the decoder. However, a very high learning rate would still destabilize training, even with a frozen encoder.

In [ ]:
# Original optimizer + scheduler
print(OmegaConf.to_yaml(char_model.cfg.optim))

In [ ]:
with open_dict(char_model.cfg.optim):
  char_model.cfg.optim.lr = 0.01
  char_model.cfg.optim.betas = [0.95, 0.5]  # from paper
  char_model.cfg.optim.weight_decay = 0.001  # Original weight decay
  char_model.cfg.optim.sched.warmup_steps = None  # Remove default number of steps of warmup
  char_model.cfg.optim.sched.warmup_ratio = 0.05  # 5 % warmup
  char_model.cfg.optim.sched.min_lr = 1e-5

### Setting up augmentation

Remember that the model was trained on several thousands of hours of data, so the regularization provided to it might not suit the current dataset. We can easily change it as we see fit.

-----

You might notice that we utilize `char_model.from_config_dict()` to create a new SpectrogramAugmentation object and assign it directly in place of the previous augmentation. This is generally the syntax to be followed whenever you notice a `_target_` tag in the config of a model's inner config. 

-----
**Note**: For low resource languages, it might be better to increase augmentation via SpecAugment to reduce overfitting. However, this might, in turn, make it too hard for the model to train in a short number of epochs.

In [ ]:
print(OmegaConf.to_yaml(char_model.cfg.spec_augment))

In [ ]:
# with open_dict(char_model.cfg.spec_augment):
#   char_model.cfg.spec_augment.freq_masks = 2
#   char_model.cfg.spec_augment.freq_width = 25
#   char_model.cfg.spec_augment.time_masks = 2
#   char_model.cfg.spec_augment.time_width = 0.05

char_model.spec_augmentation = char_model.from_config_dict(char_model.cfg.spec_augment)

## Setup Metrics

Originally, the model was trained on an English dataset corpus. When calculating Word Error Rate, we can easily use the "space" token as a separator for word boundaries. On the other hand, certain languages such as Japanese and Mandarin do not use "space" tokens, instead opting for different ways to annotate the end of the word.

In cases where the "space" token is not used to denote a word boundary, we can use the Character Error Rate metric instead, which computes the edit distance at a token level rather than a word level.

We might also be interested in noting model predictions during training and inference. As such, we can enable logging of the predictions.

In [ ]:
#@title Metric
use_cer = True #@param ["False", "True"] {type:"raw"}
log_prediction = True #@param ["False", "True"] {type:"raw"}


In [ ]:
char_model._wer.use_cer = use_cer
char_model._wer.log_prediction = log_prediction

## Setup Trainer and Experiment Manager

And that's it! Now we can train the model by simply using the Pytorch Lightning Trainer and NeMo Experiment Manager as always.

For demonstration purposes, the number of epochs is kept intentionally low. Reasonable results can be obtained in around 100 epochs (approximately 25 minutes on Colab GPUs).

In [ ]:
import torch
import pytorch_lightning as ptl

if torch.cuda.is_available():
  gpus = 1
else:
  gpus = 0

EPOCHS = 100  # 100 epochs would provide better results, but would take an hour to train

trainer = ptl.Trainer(gpus=gpus, 
                      max_epochs=EPOCHS, 
                      accumulate_grad_batches=1,
                      checkpoint_callback=False,
                      logger=False,
                      log_every_n_steps=5,
                      check_val_every_n_epoch=10)

# Setup model with the trainer
char_model.set_trainer(trainer)

# Finally, update the model's internal config
char_model.cfg = char_model._cfg

In [ ]:
# Environment variable generally used for multi-node multi-gpu training.
# In notebook environments, this flag is unnecessary and can cause logs of multiple training runs to overwrite each other.
os.environ.pop('NEMO_EXPM_VERSION', None)
LANGUAGE = "En"
config = exp_manager.ExpManagerConfig(
    exp_dir=f'experiments/lang-{LANGUAGE}/',
    name=f"ASR-Char-Model-Language-{LANGUAGE}",
    checkpoint_callback_params=exp_manager.CallbackParams(
        monitor="val_wer",
        mode="min",
        always_save_nemo=True,
        save_best_model=True,
    ),
)

config = OmegaConf.structured(config)

logdir = exp_manager.exp_manager(trainer, config)

In [ ]:
try:
  from google import colab
  COLAB_ENV = True
except (ImportError, ModuleNotFoundError):
  COLAB_ENV = False

# Load the TensorBoard notebook extension
if COLAB_ENV:
  %load_ext tensorboard
  %tensorboard --logdir /content/experiments/lang-$LANGUAGE/ASR-Char-Model-Language-$LANGUAGE/
else:
  print("To use tensorboard, please use this notebook in a Google Colab environment.")

In [ ]:
%%time
trainer.fit(char_model)

## Results

Whether you trained for small number of epochs, the character word error rate seems high (even after 100 epochs, CER is close to 45-50% or so on the test set).

Considering the limited amount of data available and the extensive vocabulary, this is expected to some degree. Remember that nearly 177 tokens were out of vocabulary in the test set - so those acoustic features can't be adequately captured using just the train set tokens. 

For reference, AISHELL datasets comprise some few hundred to one thousand hours of Mandarin speech to train roughly 5600 tokens, and those models are generally trained for a very long time on multi GPU setups.

In [ ]:
save_path = f"Model-{LANGUAGE}.nemo"
char_model.save_to(f"{save_path}")
print(f"Model saved at path : {os.getcwd() + os.path.sep + save_path}")

-----